In [1]:
import os

In [2]:
os.chdir('..')

<img src="flow_3.png">

In [3]:
from flows.flows import Flows

Welcome to the Data Science Package. First create an object as follows:
For example, use the code below to import the flow 0:
flow = Flows(0)
You can define the `categorical_threshold` which is the maximum number of categories that a categorical feature should have before considering it as continuous numeric feature. The default value is 50
For example, use the code below to import the flow 0 with defining the categorical_threshold as 50
flow = Flows(flow_id=0, categorical_threshold=50)


In [4]:
flow = Flows(3)

Please use the following function to read the data
dataframe_dict, columns_set = flow.load_data(path : str, files_list : list)
For example: path = ./data
If your data is in a nested directory, it is better to os.path.join. For example:
path = os.path.join('data', 'flow_0')
For example: files_list = ['train.csv','test.csv']
The output is a dictionary that contains dataframes e.g.

dataframe_dict = {'train': train_dataframe,'test': test_dataframe}
If you want to explore the data you can run one of the following functions: 
1 . flow.exploring_data(dataframe_dict: dict, key_i: str)
For example: flow.exploring_data(dataframe_dict, 'train')
2 . flow.comparing_statistics(dataframe_dict: dict)
For example: flow.comparing_statistics(dataframe_dict)




In [5]:
path = "./data/flow_3"
files_list = ['train_transaction.csv','test_transaction.csv']

In [6]:
dataframe_dict, columns_set = flow.load_data(path, files_list, rows_amount=1500)

A summary of the data sets


,train_transaction,test_transaction
column type,,
categorical_integer,326,325
categorical_string,14,14
continuous,54,54
date,0,0
json,0,0
other,0,0
total amount,394,393


NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
The possible ids are {'TransactionID'}
The possible possible_target are ['isFraud']
The type of the problem that should be solved {'isFraud': 'classification'}

The Kullback-Leibler Divergence between probability mass function (pmf) derived from `isFraud` and and an uniformly distributed pmf 	 = 1.257 
It is above the threshold 	 0.050. 
Imballanced target variable!


,total count,frequency (%)
isFraud values,,
0,1469,97.93
1,31,2.07


If you have categorical features with string labels, Encode the categorical features by applying the following function:
dataframe_dict, columns_set = flow.encode_categorical_feature(dataframe_dict: dict)


In [ ]:
dataframe_dict["train_transaction"].head()

In [ ]:
 columns_set["train_transaction"].keys()

In [ ]:
dataframe_dict, columns_set = flow.encode_categorical_feature(dataframe_dict,print_results=10)

In [ ]:
ignore_columns = ['isFraud']

In [ ]:
dataframe_dict, columns_set = flow.drop_columns_constant_values(dataframe_dict, ignore_columns)

In [ ]:
dataframe_dict, columns_set = flow.drop_correlated_columns(dataframe_dict, ignore_columns)

In [ ]:
ignore_columns = ["TransactionID", "isFraud"]

In [ ]:
dataframe_dict, columns_set = flow.scale_data(dataframe_dict, ignore_columns)

In [ ]:
flow.exploring_data(dataframe_dict, "test_transaction")

In [7]:
flow.comparing_statistics(dataframe_dict)

interactive(children=(IntSlider(value=0, description='feature_nr', max=393), Output()), _dom_classes=('widget-…

In [ ]:
columns = dataframe_dict["train_transaction"].columns
train_dataframe = dataframe_dict["train_transaction"][
    [x for x in columns_set["train_transaction"]["continuous"] if x not in ignore_columns]]
test_dataframe = dataframe_dict["test_transaction"][
    [x for x in columns_set["train_transaction"]["continuous"] if x not in ignore_columns]]
train_target = dataframe_dict["train_transaction"]["isFraud"]

In [ ]:
parameters = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "kfold",  # "method":"kfold"
        "fold_nr": 5,  # foldnr:5 , "split_ratios": 0.8 # "split_ratios":(0.7,0.2)
    },
    "model": {"type": "Logistic regression",
              "hyperparameters": {"alpha": "optimize",  # alpha:optimize
                                  },
              },
    "metrics": ["accuracy_score", "roc_auc_score"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

In [ ]:
model_index_list, save_models_dir, y_test = flow.training(parameters)

In [ ]:
parameters_lighgbm = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "kfold",  # "method":"kfold"
        "fold_nr": 5,  # foldnr:5 , "split_ratios": 0.8 # "split_ratios":(0.7,0.2)
    },
    "model": {"type": "lightgbm",
              "hyperparameters": dict(objective='binary', metric='cross-entropy', num_leaves=5,
                                      boost_from_average=True,
                                      learning_rate=0.05, bagging_fraction=0.99, feature_fraction=0.99, max_depth=-1,
                                      num_rounds=10000, min_data_in_leaf=10, boosting='dart')
              },
    "metrics": ["accuracy_score", "roc_auc_score"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

In [ ]:
model_index_list, save_models_dir, y_test = flow.training(parameters_lighgbm)

In [ ]:
parameters_xgboost = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "kfold",  # "method":"kfold"
        "fold_nr": 5,  # fold_nr:5 , "split_ratios": 0.3 # "split_ratios":(0.3,0.2)
    },
    "model": {"type": "xgboost",
              "hyperparameters": {'max_depth': 5, 'eta': 1, 'eval_metric': "auc", 'objective':'binary:logistic'}
              },
    "metrics": ["accuracy_score", "roc_auc_score"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

In [ ]:
model_index_list, save_models_dir, y_test = flow.training(parameters_xgboost)

In [ ]:
train_dataframe = dataframe_dict['train_transaction'].drop(['isFraud'],1)
train_target = dataframe_dict["train_transaction"]["isFraud"]

test_dataframe = dataframe_dict['test_transaction']  #.drop(['isFraud'],1)

parameters_sklearn = {
    "data": {
        "train": {
            "features": train_dataframe,
            "target": train_target.to_numpy()
        }, 
    },
    "split": {
        "method": "split",
        "split_ratios": 0.2,
        "stratify": True
    },
    "model": {
        "type": "sklearn.ensemble.RandomForestClassifier",
        "hyperparameters": {
            'params_grid':{
                'class_weight': ["balanced_subsample"],
                'max_depth': [5, 10, 20, 999],
                'min_samples_leaf': [10, 3, 1]
            },
            'params_fixed': {
                'max_depth': 4, 
                'min_samples_split': 10, 
                'min_samples_leaf': 2, 
                'random_state': 11
            },
            'params_cv': {
                'n_splits': 5, 
                'shuffle': True, 
                'random_state': 11
            },
            "objective": "classification",
            "grid_search_scoring": ['roc_auc', 'accuracy']
        },
    },
    "metrics": ['roc_auc_score', 'accuracy_score'],
    "predict": {
        "test": {
            "features": test_dataframe
        }
    }
}

In [ ]:
model_index_list, save_models_dir, y_test = flow.training(parameters_sklearn)